In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters
from sklearn.model_selection import train_test_split
import urllib
import os
import csv
import cv2
import time
from PIL import Image

from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()
sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 22, 10

RANDOM_SEED = 42

# np.random.seed(RANDOM_SEED)
# tf.random.set_seed(RANDOM_SEED)

In [2]:
from tensorflow.python.platform import gfile
with tf.Session() as sess:
    with gfile.FastGFile('snapshots/freezedmodel/1/freezed_model.pb', 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        sess.graph.as_default()
        g_in = tf.import_graph_def(graph_def)
    for op in tf.get_default_graph().get_operations():
        print(str(op.name))
    for n in tf.get_default_graph().as_graph_def().node:
        print(n.name)

Instructions for updating:
Use tf.gfile.GFile.
import/input_1
import/pyramid_regression_0/kernel
import/pyramid_regression_0/bias
import/pyramid_regression_1/kernel
import/pyramid_regression_1/bias
import/pyramid_regression_2/kernel
import/pyramid_regression_2/bias
import/pyramid_regression_3/kernel
import/pyramid_regression_3/bias
import/pyramid_regression/kernel
import/pyramid_regression/bias
import/pyramid_classification_0/kernel
import/pyramid_classification_0/bias
import/pyramid_classification_1/kernel
import/pyramid_classification_1/bias
import/pyramid_classification_2/kernel
import/pyramid_classification_2/bias
import/pyramid_classification_3/kernel
import/pyramid_classification_3/bias
import/pyramid_classification/kernel
import/pyramid_classification/bias
import/padding_conv1/Pad/paddings
import/padding_conv1/Pad
import/conv1/kernel
import/conv1/convolution/ReadVariableOp
import/conv1/convolution
import/bn_conv1/gamma
import/bn_conv1/beta
import/bn_conv1/moving_mean
import/bn_c

import/input_1
import/pyramid_regression_0/kernel
import/pyramid_regression_0/bias
import/pyramid_regression_1/kernel
import/pyramid_regression_1/bias
import/pyramid_regression_2/kernel
import/pyramid_regression_2/bias
import/pyramid_regression_3/kernel
import/pyramid_regression_3/bias
import/pyramid_regression/kernel
import/pyramid_regression/bias
import/pyramid_classification_0/kernel
import/pyramid_classification_0/bias
import/pyramid_classification_1/kernel
import/pyramid_classification_1/bias
import/pyramid_classification_2/kernel
import/pyramid_classification_2/bias
import/pyramid_classification_3/kernel
import/pyramid_classification_3/bias
import/pyramid_classification/kernel
import/pyramid_classification/bias
import/padding_conv1/Pad/paddings
import/padding_conv1/Pad
import/conv1/kernel
import/conv1/convolution/ReadVariableOp
import/conv1/convolution
import/bn_conv1/gamma
import/bn_conv1/beta
import/bn_conv1/moving_mean
import/bn_conv1/moving_variance
import/bn_conv1/ReadVariab

In [17]:
img_path = 'Images/car116.jpeg'
image = read_image_bgr(img_path)
image = preprocess_image(image.copy())
image_rs, scale = resize_image(image)

In [18]:
image_rs.shape

(800, 1067, 3)

In [19]:
image.shape

(450, 600, 3)

In [20]:
scale

1.7777777777777777

In [4]:
with tf.Session() as sess:
    with gfile.FastGFile('snapshots/freezedmodel/1/freezed_model.pb', 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        sess.graph.as_default()
        g_in = tf.import_graph_def(graph_def)
    tensor_input = sess.graph.get_tensor_by_name('import/input_1:0')
    classification_op = sess.graph.get_tensor_by_name('import/classification/concat:0')
    regression_op = sess.graph.get_tensor_by_name('import/regression/concat:0')
    predictions  = sess.run([classification_op, regression_op], \
            feed_dict={tensor_input: np.expand_dims(image, axis=0)})
    


In [5]:
print(type(predictions ))

<class 'list'>


In [7]:
len(predictions[0][0])

160956

In [8]:
len(predictions[1][0])

160956

In [14]:
predictions[1]

array([[[-0.04047905,  0.22002125, -0.20425537, -0.2957724 ],
        [ 0.11371665,  0.00930878, -0.21564615, -0.2548762 ],
        [ 0.15574035, -0.06672865, -0.28531137, -0.26610333],
        ...,
        [-0.14652328,  0.08478881, -0.03851972, -0.4488047 ],
        [-0.14787216,  0.26560202,  0.11308543, -0.48848918],
        [-0.05237148,  0.39725676,  0.17923121, -0.57494986]]],
      dtype=float32)

In [16]:
len(predictions[1][0])

160956

In [10]:
predictions[1][0][0]

array([-0.04047905,  0.22002125, -0.20425537, -0.2957724 ], dtype=float32)

In [11]:
predictions[0][0][0]

array([0.00339875], dtype=float32)

In [16]:
sorted_ = predictions[0][0]

TypeError: 'reverse' is an invalid keyword argument for sort()

In [21]:
from tensorflow.keras.models import load_model
loaded_model = load_model('snapshots/frozengraph/1/')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [22]:
image_ = np.expand_dims(image_rs, axis=0)

loaded_model.predict(image_)

AttributeError: 'AutoTrackable' object has no attribute 'predict'

In [25]:
from tensorflow.python import ops
def get_graph_def_from_file(graph_filepath):
    tf.reset_default_graph()
    with ops.Graph().as_default():
        with tf.gfile.GFile(graph_filepath, 'rb') as f:
            graph_def = tf.GraphDef()
            graph_def.ParseFromString(f.read())
    return graph_def

In [33]:
gdef =get_graph_def_from_file('modelexporter_pratik/frozen_model.pb')

In [34]:
node_names = [node.name for node in gdef.node]
print (node_names)

['input_1', 'pyramid_regression_0/kernel', 'pyramid_regression_0/bias', 'pyramid_regression_1/kernel', 'pyramid_regression_1/bias', 'pyramid_regression_2/kernel', 'pyramid_regression_2/bias', 'pyramid_regression_3/kernel', 'pyramid_regression_3/bias', 'pyramid_regression/kernel', 'pyramid_regression/bias', 'pyramid_classification_0/kernel', 'pyramid_classification_0/bias', 'pyramid_classification_1/kernel', 'pyramid_classification_1/bias', 'pyramid_classification_2/kernel', 'pyramid_classification_2/bias', 'pyramid_classification_3/kernel', 'pyramid_classification_3/bias', 'pyramid_classification/kernel', 'pyramid_classification/bias', 'padding_conv1/Pad/paddings', 'padding_conv1/Pad', 'conv1/kernel', 'conv1/convolution/ReadVariableOp', 'conv1/convolution', 'bn_conv1/gamma', 'bn_conv1/beta', 'bn_conv1/moving_mean', 'bn_conv1/moving_variance', 'bn_conv1/ReadVariableOp', 'bn_conv1/ReadVariableOp_1', 'bn_conv1/FusedBatchNormV3/ReadVariableOp', 'bn_conv1/FusedBatchNormV3/ReadVariableOp_1',